In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import math
import matplotlib.pyplot as plt
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
df = pd.read_csv("../input/flight-take-off-data-jfk-airport/M1_final.csv")

**Feature Engineering**

In [ ]:
df.head()

In [ ]:
df.info()

In [ ]:
df['OP_UNIQUE_CARRIER'].unique()

In [ ]:
df['DEST'].unique()

So, There are 6 object types in the features of our dataset. 

In [ ]:
df['Wind'].unique()

In [ ]:
df = df.drop(['Wind'],axis = 1)

In [ ]:
df['Condition'].unique()

There is some ways for splitting the Condition feature but I need to explore further. As of now, I am removing using that attribute without any changes

In [ ]:
df = df.drop(['TAIL_NUM'],axis = 1)

In [ ]:
df.info()

Label Encoding 

In [ ]:
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
enc_label=LabelEncoder()
enc_one=OneHotEncoder()
df_copy = df.copy()
error_lab = []
error_one = []
df.iloc[:,3]= enc_label.fit_transform(df.iloc[:,3])
df.iloc[:,4]= enc_label.fit_transform(df.iloc[:,4])
df.iloc[:,12]= enc_label.fit_transform(df.iloc[:,12])
df.iloc[:,17]= enc_label.fit_transform(df.iloc[:,17])
df.info()


In [ ]:
from sklearn.model_selection import train_test_split
X=df.iloc[:,:20]
y=df.iloc[:,[20]]
X_train, X_test, y_train,y_test = train_test_split(X,y,test_size=0.1, random_state=20)

Linear Regression 

In [ ]:
from sklearn.metrics import mean_squared_error
from sklearn.linear_model import LinearRegression
lr = LinearRegression()
lr.fit(X_train, y_train)
y_lr=lr.predict(X_test)
temp= math.sqrt(mean_squared_error(y_test,y_lr))
error_lab.append(temp)

Ridge Regression

In [ ]:
from sklearn.linear_model import Ridge
lr1 = Ridge(alpha=0.5, normalize = True)
lr1.fit(X_train, y_train)
y_lr1=lr1.predict(X_test)
temp= math.sqrt(mean_squared_error(y_test,y_lr1))
error_lab.append(temp)

Lasso Regression

In [ ]:
from sklearn.linear_model import Lasso
lr2 = Lasso(alpha=0.1)
lr2.fit(X_train, y_train)
y_lr2=lr2.predict(X_test)
temp= math.sqrt(mean_squared_error(y_test,y_lr2))
error_lab.append(temp)

KNN classifier

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
knn= KNeighborsClassifier(n_neighbors=20)
knn_pred= knn.fit(X_train,y_train.values.ravel()).predict(X_test)
temp= math.sqrt(mean_squared_error(y_test,knn_pred))
error_lab.append(temp)

Support Vector Regression

In [ ]:
from sklearn.svm import SVR
reg= SVR()
reg.fit(X_train,y_train.values.ravel())
svr_pred= reg.predict(X_test)
temp= math.sqrt(mean_squared_error(y_test,svr_pred))
error_lab.append(temp)

Gaussian Naive Bayes

In [ ]:
from sklearn.naive_bayes import GaussianNB
gnb= GaussianNB()
gnb_pred= gnb.fit(X_train,y_train.values.ravel()).predict(X_test)
temp= math.sqrt(mean_squared_error(y_test,gnb_pred))
error_lab.append(temp)

Random Forest Classifier

In [ ]:
from sklearn.ensemble import RandomForestClassifier
rfc= RandomForestClassifier()
rfc_pred= rfc.fit(X_train,y_train.values.ravel()).predict(X_test)
temp= math.sqrt(mean_squared_error(y_test,rfc_pred))
error_lab.append(temp)

Light BGM

In [ ]:
import lightgbm as lgb
lgb_train= lgb.Dataset(X_train, label = y_train)
lgb_eval= lgb.Dataset(X_train,y_train, reference= lgb_train)
params = {'boosting_type': 'gbdt', 'num_leaves': 60, 'learning_rate': 0.2}
gbm_pred = lgb.train(params, lgb_train, 100).predict(X_test)
temp= math.sqrt(mean_squared_error(y_test,gbm_pred))
error_lab.append(temp)

**One Hot Encoding**

In [ ]:
X.info()

In [ ]:
X_ohe = pd.get_dummies(data = X, columns=['OP_UNIQUE_CARRIER','DEST','Dew Point','Condition'])

X1_train, X1_test, y1_train, y1_test = train_test_split(X_ohe,y, test_size = 0.1, random_state =20)

Linear regression

In [ ]:
lr.fit(X1_train, y1_train)
y_lr=lr.predict(X1_test)
temp= math.sqrt(mean_squared_error(y1_test,y_lr))
error_one.append(temp)

Ridge regression

In [ ]:
lr1.fit(X1_train, y1_train)
y_lr1=lr1.predict(X1_test)
temp= math.sqrt(mean_squared_error(y1_test,y_lr1))
error_one.append(temp)

Lasso regression

In [ ]:
lr2 = Lasso(alpha=0.1)
lr2.fit(X1_train, y1_train)
y_lr2=lr2.predict(X1_test)
temp= math.sqrt(mean_squared_error(y1_test,y_lr2))
error_one.append(temp)

KNN classifier

In [ ]:
knn= KNeighborsClassifier(n_neighbors=10)
knn_pred1= knn.fit(X1_train,y1_train.values.ravel()).predict(X1_test)
temp= math.sqrt(mean_squared_error(y1_test,knn_pred1))
error_one.append(temp)

Support Vector Regression

In [ ]:
reg= SVR()
reg.fit(X1_train,y1_train.values.ravel())
svr_pred1= reg.predict(X1_test)
temp= math.sqrt(mean_squared_error(y1_test,svr_pred1))
error_one.append(temp)

Gaussian Naive Bayes

In [ ]:
gnb= GaussianNB()
gnb_pred= gnb.fit(X1_train,y1_train.values.ravel()).predict(X1_test)
temp= math.sqrt(mean_squared_error(y1_test,gnb_pred))
error_one.append(temp)

Random Forest Classifier

In [ ]:
rfc= RandomForestClassifier()
rfc_pred1= rfc.fit(X1_train,y1_train.values.ravel()).predict(X1_test)
temp= math.sqrt(mean_squared_error(y1_test,rfc_pred1))
error_one.append(temp)

Light GBM

In [ ]:
lgb_train= lgb.Dataset(X1_train, label = y1_train)
lgb_eval= lgb.Dataset(X1_train,y1_train, reference= lgb_train)
params = {'boosting_type': 'gbdt', 'num_leaves': 60, 'learning_rate': 0.2}
gbm_pred1 = lgb.train(params, lgb_train, 100).predict(X1_test)
temp= math.sqrt(mean_squared_error(y1_test,gbm_pred1))
error_one.append(temp)

**Comparing models based on RMSE**

In [ ]:
labels=['Linear Regression','Ridge Regression','Lasso Regression','KNN','SVR','Naive Bayes','Random Forest','LightGBM']
df = pd.DataFrame({'Label encoding':error_lab,'One Hot encoding':error_one}, index=labels)
ax = df.plot.bar(rot=0,figsize=(20,10))

From the above observation, we conclude that Naive Bayes Classifier has highest difference between the errors. The reason maybe because the algorithm fundamentally they are highly affected by the labeling and one hot encoding. This clearly indicates that assigning order to attribute values may not work here in this naive bayes